<a href="https://colab.research.google.com/github/ij264/Corpus-Drawing-Project/blob/master/sketchRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1831 sha256=053831fd4892284f819981e0bc51089489fe2a25444e42a6f65715b6094d1d56
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

time: 2.73 s


In [3]:
# parameters
class HyperParameters():
    def __init__(self):
        self.location = '/content/drive/My Drive/Shared drives/Corpus Drawing Project/data/sketchrnn_airplane.npz'
        self.Nz = 128
        self.batch_size = 100
        self.encoder_hidden_size = 256
        self.decoder_hidden_size = 512
        self.temperature = 0.9
        self.gradient_clipping = 1.0
        self.lr = 1e-4
        self.KL_min = 0.2
        self.R = 0.9999
        self.WKL = 1.0
        self.dropout_keep = 0.9

hp = HyperParameters()

time: 11.8 ms


In [10]:
data = np.load(hp.location, encoding='latin1', allow_pickle=True)
len(data['train'])

70000

time: 609 ms


In [ ]:
# encoder RNN
class EncoderRNN(nn.Module):
    def __init__(self):

        
        super(EncoderRNN, self).__init__()


        # bidirectional LSTM 
        self.LSTM = nn.LSTM(input_size=5, # input vector is 5x1
                            hp.encoder_hidden_size,
                            hp.decoder_hidden_size,
                            num_layers=1,
                            bias=True,
                            batch_first=False,
                            bidirectional=True
                            )
        
        self.mu = nn.Linear(in_features=hp.encoder_hidden_size,
                            hp.Nz)
        
        self.sigma = nn.Linear(in_features=hp.encoder_hidden_size,
                               hp.Nz)
        
        self.train()


    def forward(self, inputs, batch_size, hidden_cell=None):
        mu = self.mu(h)
        sigma_hat = self.sigma(h)
        sigma = torch.exp(sigma_hat/2)

In [ ]:
# NN model: a bidirectional NN with LSTM
class Model():


    def __init__(self):

        # forward encoder
        self.encoder() = EncoderRNN()

        # backward encoder
        self.decoder() = DecoderRNN()

        # TODO: implement gradient clipping
        self.encoder_optimiser() = optim.Adam(self.encoder.parameters(), lr=1e-4)
        self.decoder_optimiser() = optim.Adam(self.decoder.parameters(), lr=1e-4)


    # bivariate normal distribution probability distribution function
    def bivariate_normal_PDF(Dx, Dy):

        z = (Dx - self.mu_x)**2/self.sigma_x**2 \
        - 2 * self.rho_xy * (Dx - self.mu_x) * (Dy - self.mu_y)/(self.sigma_x * self.sigma_y) \
        + (Dy - self.mu_y)**2/self.sigma_y**2

        prefactor = 1/(2 * np.pi * self.sigma_x * self.sigma_y * torch.sqrt(1-self.rho_xy**2))

        return prefactor * torch.exp(-z/(2 * (1 - self.rho_xy**2)))


    # reconstruction loss
    def LR(self, Dx, Dy, p):

        PDF = bivariate_normal_PDF(Dx, Dy)

        LS = -1/float(N_max) * torch.sum(
            torch.log(
                torch.sum(self.Pi * PDF)
            )
        )

        LP = -1/float(N_max) * torch.sum(
            p * torch.log(self.q)
        )

        return LS + LP

    def KL(self, Dx, Dy):
